In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf


In [9]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [8]:
import tensorflow

# Dataset

In [3]:
from scipy.io import loadmat
from os.path import join

prefix = '../../Stanford_Dog_Breed'
imgfix = 'Images'
anofix = 'Annotation'

In [4]:
train = loadmat(join(prefix, 'train_list.mat'))
test = loadmat(join(prefix, 'test_list.mat'))

# L_annotation = L['annotation_list'];
# L_file = L['file_list'];
# L = L['labels'];

ftrain = train['file_list']
ftest = test['file_list']

ltrain = train['labels']
ltest = test['labels']

ntrain = ltrain.shape[0]
ntest = ltest.shape[0]

print 'Size of the training set:\t', ntrain
print 'Size of the testing set: \t', ntest

Size of the training set:	12000
Size of the testing set: 	8580


## Catagory 2 Index Table

In [5]:
DogBreed = {}
for i in range(1, ntrain):
    l = ltrain[i][0]
    c = ftrain[i][0][0].split('/')[0]
    c = c.split('-')[1]
    if c not in DogBreed:
        DogBreed[c] = l
        print c, l

Chihuahua 1
Japanese_spaniel 2
Maltese_dog 3
Pekinese 4
Shih 5
Blenheim_spaniel 6
papillon 7
toy_terrier 8
Rhodesian_ridgeback 9
Afghan_hound 10
basset 11
beagle 12
bloodhound 13
bluetick 14
black 15
Walker_hound 16
English_foxhound 17
redbone 18
borzoi 19
Irish_wolfhound 20
Italian_greyhound 21
whippet 22
Ibizan_hound 23
Norwegian_elkhound 24
otterhound 25
Saluki 26
Scottish_deerhound 27
Weimaraner 28
Staffordshire_bullterrier 29
American_Staffordshire_terrier 30
Bedlington_terrier 31
Border_terrier 32
Kerry_blue_terrier 33
Irish_terrier 34
Norfolk_terrier 35
Norwich_terrier 36
Yorkshire_terrier 37
wire 38
Lakeland_terrier 39
Sealyham_terrier 40
Airedale 41
cairn 42
Australian_terrier 43
Dandie_Dinmont 44
Boston_bull 45
miniature_schnauzer 46
giant_schnauzer 47
standard_schnauzer 48
Scotch_terrier 49
Tibetan_terrier 50
silky_terrier 51
soft 52
West_Highland_white_terrier 53
Lhasa 54
flat 55
curly 56
golden_retriever 57
Labrador_retriever 58
Chesapeake_Bay_retriever 59
German_short 60


In [6]:
def gen_path(idx):
    try:    path = join(prefix, imgfix, ftest[idx][0][0])
    except: path = join(prefix, imgfix, ftest[-1][0][0])      
    return  path

In [7]:
def gen_labl(idx):
    try:    labl = ltest[idx][0]
    except: labl = ltest[-1][0]
    return  labl

# Models

In [8]:
import tensorflow.keras.applications.resnet50 as resnet50 
import tensorflow.keras.applications.vgg19 as vgg19
import tensorflow.keras.applications.densenet as densenet
import tensorflow.keras.applications.inception_v3 as inception_v3

In [9]:
import tensorflow.keras.preprocessing.image as image

## Data Preparation

In [10]:
X = []
for idx in range(ntrain):
    path = join(prefix, imgfix, ftrain[idx][0][0])
    try:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        X.append(resnet50.preprocess_input(x))
    except:
        print 'Error at processing the', idx, 'images.' 

XTrain = np.vstack(X)

In [ ]:
X = []
for idx in range(ntest):
    path = gen_path(idx)
    try:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        X.append(resnet50.preprocess_input(x))
    except:
        print 'Error at processing the', idx, 'images.' 
        
XTest = np.vstack(X)

In [46]:
print(XTrain.shape)
print(XTest.shape)

(8580, 224, 224, 3)


## Classify using ImageNet Classes with ResNet50

In [ ]:
resnet50_model = resnet50.ResNet50(weights='imagenet')


In [34]:
vgg19_naive = vgg19.VGG19(weights='imagenet')
vgg19_model = tensorflow.keras.models.Model(inputs=vgg19_naive.input, 
                                            outputs=vgg19_naive.get_layer('block4_pool').output)

In [52]:
X1 = vgg19.preprocess_input(x)
X2 = resnet50.preprocess_input(x)
X3 = densenet.preprocess_input(x)
X4 = inception_v3.preprocess_input(x)

In [55]:
print np.sum(X2 - X1)
print np.sum(X3 - X2)
print np.sum(X4 - X3)

0.0
0.0
0.0


X

In [ ]:
X